importing library

In [12]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests

Make a GET request to fetch the raw HTML content


In [13]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text


# Step 2: Parse the html content
soup = BeautifulSoup(html_content)
#print(soup.prettify()) # print the parsed data of html



#Get the table having the class wikitable
table = soup.find("table", attrs={"class": "wikitable"})
table_data = table.tbody.find_all("tr")  # contains 2 rows
print ( len(table_data))






288


putiting to data frame

In [14]:
# Get all the headings 
headings = []
for th in table_data[0].find_all("th"):
    headings.append(th.text)

#tranforming to dictionary 
data={}
for i in range(1,len(table_data)): 
    content=[]
    for td in table_data[i].find_all("td"):
        content.append(td.text)
    data.setdefault(headings[0],[]).append(content[0])
    data.setdefault(headings[1],[]).append(content[1])
    data.setdefault(headings[2],[]).append(content[2])

# 
data=pd.DataFrame.from_dict(data)
# remove /n
data = data.replace('\n',' ', regex=True)

# delete rows with 'Not assigned' in the 'Borough' coloumn
indexNames = data[ data['Borough'] == 'Not assigned' ].index
data.drop(indexNames , inplace=True)

# repalce if Neighbourhood has 'Not assigned'
indexNames = data[ data['Neighbourhood\n'].str.strip() == 'Not assigned' ].index
data['Neighbourhood\n'][indexNames]=data['Borough'][indexNames]
#




combine two Neighbourhood with the same Postcode and Borough 
and make into separate columns

In [19]:

data1=data.groupby(['Postcode','Borough'],as_index = True)['Neighbourhood\n'].apply(lambda x: pd.Series(list(x))).unstack().reset_index()

data1.columns=['Postcode','Borough','Neighbourhood1','Neighbourhood2','Neighbourhood3','Neighbourhood4','Neighbourhood5','Neighbourhood6','Neighbourhood7','Neighbourhood8']
data1['Neighbourhoodall'] = data1[['Neighbourhood1', 'Neighbourhood2','Neighbourhood3','Neighbourhood4','Neighbourhood5','Neighbourhood6','Neighbourhood7','Neighbourhood8']].apply(lambda x: ','.join(x.fillna('').map(str)), axis=1)
data1['Neighbourhoodall'] = data1['Neighbourhoodall'].str.strip(',')
data1['Neighbourhoodall']
datawithcomma = data1[['Postcode','Borough','Neighbourhoodall']].copy()
datawithcomma['Borough']



0           Scarborough
1           Scarborough
2           Scarborough
3           Scarborough
4           Scarborough
5           Scarborough
6           Scarborough
7           Scarborough
8           Scarborough
9           Scarborough
10          Scarborough
11          Scarborough
12          Scarborough
13          Scarborough
14          Scarborough
15          Scarborough
16          Scarborough
17           North York
18           North York
19           North York
20           North York
21           North York
22           North York
23           North York
24           North York
25           North York
26           North York
27           North York
28           North York
29           North York
             ...       
73                 York
74                 York
75     Downtown Toronto
76         West Toronto
77         West Toronto
78         West Toronto
79           North York
80                 York
81                 York
82         West Toronto
83         West 

# I tried a lot this link but unfortuantely it does not wrok, even it cannot be donlowded


In [ ]:
import csv
import urllib2

url = 'http://cocl.us/Geospatial_data'
response = urllib2.urlopen(url)

geo= csv.reader(response, delimiter=',')

# So in here I decided to use geopy library to assing latitude and longitude to every Borough
Sorry I did not have any other choice

In [36]:
from geopy.geocoders import Nominatim


borough =  datawithcomma['Borough'] 
neighborhood_name = datawithcomma['Neighbourhoodall']
postcode=datawithcomma['Postcode']
lan=[]
lon=[]
geolocator = Nominatim(user_agent="ny_explorer")
for i in range(len(datawithcomma)):

    
    location = geolocator.geocode(datawithcomma['Borough'][i])
    lan.append(location.latitude)
    lon.append(location.longitude)
    
    
neighborhoods = neighborhoods.append({ 'postcode': postcode,
                                         'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': lan,
                                          'Longitude': lon}, ignore_index=True)
neighborhoods


GeocoderTimedOut: Service timed out

GeocoderTimedOut: Service timed out

NameError: name 'neighborhoods' is not defined